In [82]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

df = pd.DataFrame({'item': ['TV', 'Phone', 'Laptop', 'TV', 'Phone'],
                   'grade': ['A', 'C', 'B', 'B', 'A'], # 순서가 있는 변수
                   'status': ['new', 'used', 'new', 'refurbished', 'used'] # 순서가 없는 변수
                  })

In [83]:
# 순서형 변수 'grade'에 적용
encoder_label = LabelEncoder()
df['grade_encoded'] = encoder_label.fit_transform(df['grade'])

# 매핑 관계 확인
print("--- LabelEncoder Classes ---")
print(encoder_label.classes_) # ['A', 'B', 'C']

print("\n--- LabelEncoder Result ---")
print(df[['grade', 'grade_encoded']])
'''
--- LabelEncoder Result ---
  grade  grade_encoded
0     A              0
1     C              2
2     B              1
3     B              1
4     A              0
'''

--- LabelEncoder Classes ---
['A' 'B' 'C']

--- LabelEncoder Result ---
  grade  grade_encoded
0     A              0
1     C              2
2     B              1
3     B              1
4     A              0


In [84]:
# 'status' 열에 대해 원-핫 인코딩 수행
dummies = pd.get_dummies(df['status'], prefix='status')

# 원본 데이터프레임과 합치기
df_dummies = pd.concat([df, dummies], axis=1)

print("--- pd.get_dummies() Result ---")
print(df_dummies)

# 다중공선성 방지를 위해 첫 번째 카테고리 열 제거
dummies_dropped = pd.get_dummies(df['status'], prefix='status', drop_first=True)
print("\n--- with drop_first=True ---")
print(dummies_dropped)

--- pd.get_dummies() Result ---
     item grade       status  grade_encoded  status_new  status_refurbished  \
0      TV     A          new              0           1                   0   
1   Phone     C         used              2           0                   0   
2  Laptop     B          new              1           1                   0   
3      TV     B  refurbished              1           0                   1   
4   Phone     A         used              0           0                   0   

   status_used  
0            0  
1            1  
2            0  
3            0  
4            1  

--- with drop_first=True ---
   status_refurbished  status_used
0                   0            0
1                   0            1
2                   0            0
3                   1            0
4                   0            1


In [89]:
# 1. 인코더 객체 생성
# handle_unknown='ignore': 테스트 데이터에 새로운 카테고리가 나타나면 모두 0으로 처리
encoder_ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

# 2. 데이터에 fit_transform 적용
ohe_transformed = encoder_ohe.fit_transform(df[['status']])

# 3. 결과를 데이터프레임으로 변환
df_ohe = pd.DataFrame(ohe_transformed, columns=encoder_ohe.get_feature_names())

print("--- OneHotEncoder Result ---")
print(df_ohe)

--- OneHotEncoder Result ---
   x0_new  x0_refurbished  x0_used
0     1.0             0.0      0.0
1     0.0             0.0      1.0
2     1.0             0.0      0.0
3     0.0             1.0      0.0
4     0.0             0.0      1.0


In [90]:
import pandas as pd

df = pd.DataFrame({
    'item': ['TV', 'Phone', 'Laptop', 'TV', 'Phone'],
    'grade': ['A', 'C', 'B', 'B', 'A'],
    'status': ['new', 'used', 'new', 'refurbished', 'used'],
    'target': [100, 200, 150, 120, 180]   # 예시 타겟 변수
})

# status별 target 평균값으로 매핑
mean_map = df.groupby('status')['target'].mean()
df['status_target_enc'] = df['status'].map(mean_map)

print(df[['status', 'status_target_enc']])

        status  status_target_enc
0          new                125
1         used                190
2          new                125
3  refurbished                120
4         used                190


In [81]:
pd.DataFrame(scaler_robust.inverse_transform(df_robust)).astype("int")

,0,1
0,50000,25
1,52000,30
2,48000,28
3,150000,45
4,51000,26
5,49000,27
6,1000000,35
